<a href="https://colab.research.google.com/github/GH-6021/face-emotion-recognition/blob/main/src/mobileNet_fineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import google.protobuf

print("TensorFlow version:", tf.__version__)
print("protobuf version:", google.protobuf.__version__)


TensorFlow version: 2.18.0
protobuf version: 5.29.3


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# 삭제할 폴더들이 있는 상위 디렉터리 (Colab 환경에 맞춰서 변경)
base_dir = "/content/train/Training"

# 감정 폴더 리스트
emotion_folders = ["Angry", "Disgust", "Fear", "Happiness", "Neutral", "Sadness", "Surprise"]

# 각 폴더를 순회하며 파일 개수를 조정
for emotion in emotion_folders:
    folder_path = os.path.join(base_dir, emotion)

    if not os.path.exists(folder_path):
        print(f"폴더 없음: {folder_path}")
        continue

    # 폴더 내 모든 파일 목록 가져오기 (경로 포함)
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')]

    # 파일이 5000개 이하이면 삭제할 필요 없음
    if len(files) <= 10000:
        print(f"{emotion} 폴더는 이미 5000개 이하입니다. ({len(files)}개)")
        continue

    # 파일을 생성 시간 순으로 정렬 (오래된 파일 먼저 삭제)
    files.sort(key=os.path.getctime)

    # 초과하는 파일 개수 계산
    excess_files = len(files) - 10000

    # 초과된 파일 삭제
    for file in files[:excess_files]:
        os.remove(file)

    print(f"{emotion} 폴더에서 {excess_files}개의 파일을 삭제 완료! 현재 남은 파일: 5000개")


Angry 폴더는 이미 5000개 이하입니다. (10000개)
Disgust 폴더는 이미 5000개 이하입니다. (10000개)
Fear 폴더는 이미 5000개 이하입니다. (10000개)
Happiness 폴더는 이미 5000개 이하입니다. (10000개)
Neutral 폴더는 이미 5000개 이하입니다. (10000개)
Sadness 폴더는 이미 5000개 이하입니다. (10000개)
Surprise 폴더에서 1311개의 파일을 삭제 완료! 현재 남은 파일: 5000개


In [3]:
!unzip /content/drive/MyDrive/validation.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: validation/Surprise/56290c89488ebabdc514d446aa50e91434c3c7779fa2fc96d995e6b6dbb99b41_m_20_Surprise_20210215181814-010-005.jpg  
  inflating: validation/Surprise/562b0c37ce1012cb75c4ffa7bc2be5f71f978ec28f7bd6648e94f194c1e63bfe_w_40_Surprise_20210225162838-002-003.jpg  
  inflating: validation/Surprise/562b0c37ce1012cb75c4ffa7bc2be5f71f978ec28f7bd6648e94f194c1e63bfe_w_40_Surprise_20210225162858-010-001.jpg  
  inflating: validation/Surprise/564fa5b52c3c9dc04b6d485c8cd018e510f3a5d45eab364d1963979ce0455ac6_w_30_Surprise_20201207170658-003-029.jpg  
  inflating: validation/Surprise/564fa5b52c3c9dc04b6d485c8cd018e510f3a5d45eab364d1963979ce0455ac6_w_30_Surprise_20201207170658-003-043.jpg  
  inflating: validation/Surprise/564fa5b52c3c9dc04b6d485c8cd018e510f3a5d45eab364d1963979ce0455ac6_w_30_Surprise_20201207170702-004-001.jpg  
  inflating: validation/Surprise/564fa5b52c3c9dc04b6d485c8cd018e510f3a5d45eab364d1963979ce0455ac6_w_30_Surprise_202012

In [4]:
!unzip /content/drive/MyDrive/train.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144756-002-017.jpg  
  inflating: train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144756-002-018.jpg  
  inflating: train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144820-003-001.jpg  
  inflating: train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144820-003-002.jpg  
  inflating: train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144820-003-003.jpg  
  inflating: train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618dc47f13_m_30_Surprise_20201207144820-003-004.jpg  
  inflating: train/Training/Surprise/ec894aea1f2e215a6626ac9a3ae6b16f391d337d509dc411f80656618

In [ ]:
!unzip -j /content/drive/MyDrive/train.zip "train/Training/Surprise/*" -d /content/train/Training/Surprise


In [ ]:
import shutil

# 삭제할 폴더 경로
folder_path = '/content/train/Training/Happiness'

# 폴더 및 그 안의 모든 파일 삭제
shutil.rmtree(folder_path)


In [3]:
from tensorflow.keras.models import load_model

# 기존 모델 로드
model = load_model("/content/mobilenet_7.h5")

# 모델 구조 확인
model.summary()


Model: "model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)                 │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_pad (ZeroPadding2D)            │ (None, 225, 225, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 112, 112, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_bn (BatchNormalization)        │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_relu (ReLU)                    │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1 (DepthwiseConv2D)          │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_bn (BatchNormalization)    │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_relu (ReLU)                │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1 (Conv2D)                   │ (None, 112, 112, 64)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_bn (BatchNormalization)    │ (None, 112, 112, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_relu (ReLU)                │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pad_2 (ZeroPadding2D)           │ (None, 113, 113, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2 (DepthwiseConv2D)          │ (None, 56, 56, 64)          │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_bn (BatchNormalization)    │ (None, 56, 56, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_relu (ReLU)                │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2 (Conv2D)                   │ (None, 56, 56, 128)         │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3 (DepthwiseConv2D)          │ (None, 56, 56, 128)         │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 3,493,063 (13.32 MB)

 Trainable params: 3,471,175 (13.24 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

conv_output = model.get_layer('conv_pw_13_relu').output
x = GlobalAveragePooling2D()(conv_output)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
num_classes = 7
predictions = Dense(num_classes, activation='softmax')(x)

# 새로운 모델 정의
fine_tuned_model = Model(inputs=model.input, outputs=predictions)

# 일부 레이어만 동결 해제 (마지막 50개 레이어 학습 가능)
for layer in model.layers[:-50]:  # 처음 ~ (끝에서 50개 이전) 레이어는 동결
    layer.trainable = False
for layer in model.layers[-50:]:  # 마지막 50개 레이어는 학습 가능
    layer.trainable = True

# 모델 컴파일 (learning_rate 작게 설정)
fine_tuned_model.compile(optimizer=Adam(learning_rate=1e-5),  # 작은 학습률로 미세 조정
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

fine_tuned_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)                 │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_pad (ZeroPadding2D)            │ (None, 225, 225, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 112, 112, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_bn (BatchNormalization)        │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_relu (ReLU)                    │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1 (DepthwiseConv2D)          │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_bn (BatchNormalization)    │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_relu (ReLU)                │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1 (Conv2D)                   │ (None, 112, 112, 64)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_bn (BatchNormalization)    │ (None, 112, 112, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_relu (ReLU)                │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pad_2 (ZeroPadding2D)           │ (None, 113, 113, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2 (DepthwiseConv2D)          │ (None, 56, 56, 64)          │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_bn (BatchNormalization)    │ (None, 56, 56, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_relu (ReLU)                │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2 (Conv2D)                   │ (None, 56, 56, 128)         │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3 (DepthwiseConv2D)          │ (None, 56, 56, 128)         │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 3,360,967 (12.82 MB)

 Trainable params: 3,202,055 (12.21 MB)

 Non-trainable params: 158,912 (620.75 KB)

In [2]:
import tensorflow as tf

# TPU 초기화
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("✅ TPU 초기화 완료")
except ValueError:
    print("❌ TPU를 찾을 수 없습니다. 런타임 설정을 확인하세요.")
    strategy = tf.distribute.get_strategy()  # GPU/CPU fallback


✅ TPU 초기화 완료


In [7]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ TPU 설정
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU 확인
    if not tf.config.list_logical_devices('TPU'):  # TPU가 아직 초기화되지 않았다면
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("✅ TPU 연결 성공")
except ValueError:
    strategy = tf.distribute.get_strategy()  # TPU가 없으면 GPU/CPU 사용
    print("⚠️ TPU를 찾을 수 없습니다. GPU/CPU로 실행합니다.")

print("💡 실행 전략:", strategy)

# ✅ 데이터 로드 & 전처리
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32  # TPU는 큰 배치 크기를 추천함

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    "/content/train/Training",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    "/content/validation",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

# `steps_per_epoch`로 한 에폭에 사용되는 배치 수 제한
# 예시: 8000개의 샘플만 사용 (배치 사이즈가 32일 경우)
steps_per_epoch = 8000 // 32  # 배치 사이즈에 맞춰 계산
validation_steps = 200 // 32  # 검증 데이터에서 사용할 샘플 수


# ✅ TPU에서 모델 로드
with strategy.scope():
    model = load_model("/content/mobilenet_7.h5")

    # 마지막 몇 개의 층만 학습 가능하도록 설정 (Fine-Tuning)
    for layer in model.layers[:-5]:  # 마지막 5개 층만 학습
        layer.trainable = False

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

model.summary()

# ✅ 모델 학습
EPOCHS = 10

# ✅ 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# ✅ 모델 저장
model.save("/content/mobilenet_finetuned_tpu.h5")
print("✅ Fine-Tuning 완료 및 모델 저장됨!")


✅ TPU 연결 성공
💡 실행 전략: <tensorflow.python.distribute.tpu_strategy.TPUStrategyV2 object at 0x788afea64410>
Found 83138 images belonging to 7 classes.
Found 44625 images belonging to 7 classes.


Model: "model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)                 │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_pad (ZeroPadding2D)            │ (None, 225, 225, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 112, 112, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_bn (BatchNormalization)        │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_relu (ReLU)                    │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1 (DepthwiseConv2D)          │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_bn (BatchNormalization)    │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_relu (ReLU)                │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1 (Conv2D)                   │ (None, 112, 112, 64)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_bn (BatchNormalization)    │ (None, 112, 112, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_relu (ReLU)                │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pad_2 (ZeroPadding2D)           │ (None, 113, 113, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2 (DepthwiseConv2D)          │ (None, 56, 56, 64)          │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_bn (BatchNormalization)    │ (None, 56, 56, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_relu (ReLU)                │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2 (Conv2D)                   │ (None, 56, 56, 128)         │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3 (DepthwiseConv2D)          │ (None, 56, 56, 128)         │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 3,493,063 (13.32 MB)

 Trainable params: 264,199 (1.01 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 363s 1s/step - accuracy: nan - loss: nan - val_accuracy: 0.2071 - val_loss: 1.9348
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 343s 1s/step - accuracy: nan - loss: nan - val_accuracy: 0.2174 - val_loss: 1.9585
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 339s 1s/step - accuracy: nan - loss: nan - val_accuracy: 0.2579 - val_loss: 1.9492
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 337s 1s/step - accuracy: nan - loss: nan - val_accuracy: 0.2990 - val_loss: 1.9470
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 330s 1s/step - accuracy: nan - loss: nan - val_accuracy: 0.0972 - val_loss: 1.9528
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 333s 1s/step - accuracy: nan - loss: nan - val_accuracy: 0.1623 - val_loss: 1.9540
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 328s 1s/step - accuracy: nan - loss: nan - val_accuracy: 0.2422 - val_loss: 1.9345
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 322s 1s/step - accuracy: nan - loss: nan - val_accuracy: 0.1836 - val_loss: 1.9485
Epoch 9/10
250/2

✅ Fine-Tuning 완료 및 모델 저장됨!


/////////

In [ ]:
# from PIL import Image, ExifTags
# import os

# def clean_exif_and_save(image_path):
#     """ 손상된 EXIF 데이터를 제거하고 이미지 저장 """
#     try:
#         with Image.open(image_path) as img:
#             img_without_exif = img.copy()
#             img_without_exif.info.pop("exif", None)  # EXIF 제거
#             img_without_exif.save(image_path)  # 다시 저장
#     except Exception as e:
#         print(f"이미지 처리 중 오류 발생: {image_path} -> {e}")

# def process_directory(directory):
#     """ 디렉토리 내 모든 이미지의 EXIF 데이터를 정리 """
#     for subdir, _, files in os.walk(directory):
#         for file in files:
#             file_path = os.path.join(subdir, file)
#             clean_exif_and_save(file_path)

# process_directory("D:/train/Training")
# process_directory("C:/baekjoon_Git_repository/face-emotion-recognition/src/mobileNet_fineTuning_dataset/validation")


In [ ]:
# import os
# from PIL import Image

# def remove_corrupt_images(directory):
#     for subdir, _, files in os.walk(directory):
#         for filename in files:
#             file_path = os.path.join(subdir, filename)
#             try:
#                 with Image.open(file_path) as img:
#                     img.verify()  # 첫 번째 검증 (메타데이터 확인)
#                 with Image.open(file_path) as img:
#                     img.convert("RGB")  # 두 번째 검증 (실제 픽셀 데이터 확인)
#             except (OSError, IOError):
#                 print(f"손상된 이미지 삭제: {file_path}")
#                 os.remove(file_path)

# train_dir = 'D:/train/Training'
# valid_dir = 'C:/baekjoon_Git_repository/face-emotion-recognition/src/mobileNet_fineTuning_dataset/validation'

# remove_corrupt_images(train_dir)
# remove_corrupt_images(valid_dir)


/////////

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 증강 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.8  # 학습 데이터의 10%만 사용
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "/content/train/Training"
valid_dir = "/content/validation"

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical',
    shuffle=True,
    subset='training'  # 학습 데이터를 일부만 사용
)

val_generator = val_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# 모델 학습
history = fine_tuned_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Found 16631 images belonging to 7 classes.
Found 44625 images belonging to 7 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipy

In [9]:
print("Train classes:", train_generator.class_indices)
print("Validation classes:", val_generator.class_indices)


Train classes: {'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happiness': 3, 'Neutral': 4, 'Sadness': 5, 'Surprise': 6}
Validation classes: {'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happiness': 3, 'Neutral': 4, 'Sadness': 5, 'Surprise': 6}


In [ ]:
#모델저장
fine_tuned_model.save('fine_tuned_model.h5')

In [ ]:
#모델평가
loss, accuracy = fine_tuned_model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")